In [2]:
%pip install torch 
%pip install PIL
%pip install facenet-pytorch

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 8.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 10.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 26.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 29.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 13.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 7.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 19.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 24.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 24.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 17.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━

In [3]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from PIL import Image

In [3]:
# Initialize MTCNN for face detection
mtcnn = MTCNN(keep_all=True)

# Load pre-trained FaceNet model
resnet = InceptionResnetV1(pretrained='casia-webface').eval()

# Load an image containing faces
img = Image.open('../../../../../cds-vis-data/newspapers/GDL/GDL-1798-02-05-a-p0001.jpg') #0 faces
img_2 = Image.open('../../../../../cds-vis-data/newspapers/GDL/GDL-1987-08-21-a-p0012.jpg') #1 face 
img_3 =  Image.open('../../../../../cds-vis-data/newspapers/GDL/GDL-1997-08-09-a-p0021.jpg') #4 faces


# Detect faces in the image
boxes, _ = mtcnn.detect(img)

  0%|          | 0.00/111M [00:00<?, ?B/s]

In [4]:
print(boxes)

None


In [5]:

#img_2 contains 1 face and returns 1 list with 1 of 4 numbers: [[594.2079467773438 1372.92529296875 678.6680908203125 1482.1539306640625]]
#img_3 contains 4 faces and return 1 list with 4 list 

#to get the number of faces access the first element in boxes
boxes.shape[0]

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
#method 1:


# Initialize MTCNN for face detection:
#mtcnn = MTCNN(keep_all=True)

# Load pre-trained FaceNet model:
#resnet = InceptionResnetV1(pretrained='casia-webface').eval()


# Initialize dictionaries:
#faces_rawcount_per_decade = {}  # Dictionary to store raw counts of images with faces per decade for each newspaper
#perc_pages_with_faces_per_decade = {}  # Dictionary to store percentage of pages with faces per decade for each newspaper


#define folderpath:
#folderpath = ....

#create a csv for each of the 3 newspapers: 
#loop through each subfolder (1 subfolder=1 newspaper) in folderpath

    #extract name of newspaper:
    #newpaper_name = extract name of the subfolder

    #loop thoguh each file in subfolder:

        ##to get decade: 

        # load in image:
        #img= Image.open(path_to_image)

        # get filename:
        #filename = what comes after the last '/' in path_to_image

        #between the first '-' and second '-' in filename extract 3 first numbers:
        #3_numbers = 

        #add a 0 to the end: 
        #4_numbers= 3_numbers + "0"

        #add every unique instances to faces_rawcount_per_decade dictiory: 
        #if 4_numbers is not allready in faces_rawcount_per_decade, add it to the list
            #else skip
        
        #for each element in decades dictiory, count the rawcount of faces:


            #detect faces 
            #boxes, _ = mtcnn.detect(img)
        

            #if shape of boxes == Nonce
                #add 0 to faces_on_page

                #else
                    #number_to_add = boxes.shape[0]
                    #add number_to_add to faces_on_page_count
            
    
        






In [4]:
%pip install pandas
%pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 44.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 12.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 4.5 MB/s eta 0:00:00:00:01

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.5/159.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 37.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 36.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.9 MB/s et

In [7]:
#obs took about 7 min, for 3*8 files (machine 4)

#new, WORKING BUT NOT QUITE RIGHT 
import os
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from facenet_pytorch import MTCNN

# Initialize MTCNN for face detection:
mtcnn = MTCNN()

# Define folder path
folderpath = "../newspapers_2/"

# Initialize dictionaries to store results:
faces_rawcount_per_decade = {}  # Store raw counts of images with faces per decade for each newspaper
perc_pages_with_faces_per_decade = {}  # Store percentage of pages with faces per decade for each newspaper

# Function to extract decade from filename:
def extract_decade(filename):
    parts = filename.split("-") #plits the filename into parts seperated by -
    year = int(parts[1])  # Extract year from filename, be taking the second part of the filename and making if from string to integer
    decade = (year // 10) * 10  #year is devided by 10 and the integer division // gets rid of any decimals, it is then multiplied by 10 to get the 0 as last number
    return decade

# Iterate through each subfolder (newspaper) in folderpath:
for newspaper_folder in os.listdir(folderpath):

    # Construct the path to the newspaper we are looping through 
    full_path = os.path.join(folderpath, newspaper_folder)

    # Check if the path full_path exist as a directory, and will procede if it is 
    if os.path.isdir(full_path):

        # Extract the name of the newspaper from the folder path
        newspaper_name = newspaper_folder.upper() #obs all are upper, so could delete that

        # Initialize dictionaries for the current newspaper
        faces_rawcount_per_decade[newspaper_name] = {}
        perc_pages_with_faces_per_decade[newspaper_name] = {}
        
        # Iterate through each file in the subfolder (newspaper):
        for filename in os.listdir(full_path):

            # Construct the full file path
            file_path = os.path.join(full_path, filename)

            # Check if the current item is a file and ends with ".jpg", because there is a readme file in the newspaper folder
            if os.path.isfile(file_path) and filename.endswith(".jpg"):

                # Proceed with processing the image file
                # Open the image file
                img = Image.open(file_path)

                # Extract the decade from the filename
                decade = extract_decade(filename)
                
                # Detect faces in the image:
                boxes, _ = mtcnn.detect(img)


                # Count faces per decade:
                # Check if any face is detected
                if boxes is not None:
                    num_faces = boxes.shape[0]  # Count the number of detected faces
                else:
                    num_faces = 0  # No faces detected

                #make a varible for whether the the file contains at least 1 image (for calculating %)
                #if num_faces = 0
                    #assign is_face_present = 0
                #if num_faces != 0
                    #assign is_face_present = 1
                
                # Update the count of faces for the current decade, based on wheter the decade is allready there
                if decade not in faces_rawcount_per_decade[newspaper_name]:
                    faces_rawcount_per_decade[newspaper_name][decade] = num_faces #if not there, add the number as the first one
                else:
                    faces_rawcount_per_decade[newspaper_name][decade] += num_faces #if there, add the number to what is allready there



        # now for calculating the percentage of pages with faces per decade:
        # Calculate the total number of pages in the current newspaper
        total_pages = len([filename for filename in os.listdir(full_path) if filename.endswith(".jpg")]) 
        
        # Iterate through each decade and calculate the percentage of pages with faces:

        #iterates over each key-value pair in the faces_rawcount_per_decade[newspaper_name] dictionary.
        for decade, faces_count in faces_rawcount_per_decade[newspaper_name].items():
            
            # Calculate the percentage
            percentage = (faces_count / total_pages) * 100 
            # Store the percentage in the dictionary
            perc_pages_with_faces_per_decade[newspaper_name][decade] = percentage


    #save results to df
    for newspaper_name, data in faces_rawcount_per_decade.items():
        df = pd.DataFrame.from_dict(data, orient='index', columns=['Faces_Count'])
        df.to_csv(f'../out/{newspaper_name}_faces_per_decade.csv')





# Save results into CSV files:
#for newspaper_name, data in faces_rawcount_per_decade.items():
    #df = pd.DataFrame.from_dict(data, orient='index', columns=['Faces_Count'])
    #df.to_csv(f'../out/{newspaper_name}_faces_per_decade.csv')

#for newspaper_name, data in perc_pages_with_faces_per_decade.items():
    #df = pd.DataFrame.from_dict(data, orient='index', columns=['Percentage'])
    #df.to_csv(f'..out/{newspaper_name}_percentage_per_decade.csv')




In [11]:
file_path

'../newspapers_2/GDL/GDL-1798-02-05-a-p0001.jpg'

In [8]:
perc_pages_with_faces_per_decade

{'IMP': {2010: 137.5, 1880: 0.0},
 'JDG': {1990: 25.0, 1820: 0.0},
 'GDL': {1990: 12.5, 1790: 0.0}}

In [12]:
for newspaper_name, data in faces_rawcount_per_decade.items():
        df = pd.DataFrame.from_dict(data, 
        orient='index', columns=['Faces_Count'])

        df.to_csv(f'../out/{newspaper_name}_faces_per_decade.csv')

In [9]:
faces_rawcount_per_decade

{'IMP': {2010: 11, 1880: 0},
 'JDG': {1990: 2, 1820: 0},
 'GDL': {1990: 1, 1790: 0}}

In [14]:
#NEW, FIXING PERCENTAGE


import os
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from facenet_pytorch import MTCNN

# Initialize MTCNN for face detection:
mtcnn = MTCNN()

# Define folder path
folderpath = "../newspapers_2/"

# Initialize dictionaries to store results:
faces_rawcount_per_decade = {}  # Store raw counts of images with faces per decade for each newspaper
perc_pages_with_faces_per_decade = {}  # Store percentage of pages with faces per decade for each newspaper

# Function to extract decade from filename:
def extract_decade(filename):
    parts = filename.split("-") #plits the filename into parts seperated by -
    year = int(parts[1])  # Extract year from filename, be taking the second part of the filename and making if from string to integer
    decade = (year // 10) * 10  #year is devided by 10 and the integer division // gets rid of any decimals, it is then multiplied by 10 to get the 0 as last number
    return decade

# Iterate through each subfolder (newspaper) in folderpath:
for newspaper_folder in os.listdir(folderpath):

    # Construct the path to the newspaper we are looping through 
    full_path = os.path.join(folderpath, newspaper_folder)

    # Check if the path full_path exist as a directory, and will procede if it is 
    if os.path.isdir(full_path):

        # Extract the name of the newspaper from the folder path
        newspaper_name = newspaper_folder.upper() #obs all are upper, so could delete that

        # Initialize dictionaries for the current newspaper
        faces_rawcount_per_decade[newspaper_name] = {}
        perc_pages_with_faces_per_decade[newspaper_name] = {}
        
        # Initialize counter for pages with faces for the current decade
        pages_with_faces = 0

        # Iterate through each file in the subfolder (newspaper):
        for filename in os.listdir(full_path):

            # Construct the full file path
            file_path = os.path.join(full_path, filename)

            # Check if the current item is a file and ends with ".jpg", because there is a readme file in the newspaper folder
            if os.path.isfile(file_path) and filename.endswith(".jpg"):

                # Proceed with processing the image file
                # Open the image file
                img = Image.open(file_path)

                # Extract the decade from the filename
                decade = extract_decade(filename)
                
                # Detect faces in the image:
                boxes, _ = mtcnn.detect(img)


                # Count faces per decade:
                # Check if any face is detected
                if boxes is not None:
                    num_faces = boxes.shape[0]  # Count the number of detected faces
                else:
                    num_faces = 0  # No faces detected

                #make a varible for whether the the file contains at least 1 image (for calculating % later)
                if boxes is not None and len(num_faces) > 0:
                # Increment the counter if at least one face is detected on this page
                pages_with_faces += 1
                
                # Update the count of faces for the current decade, based on wheter the decade is allready there
                if decade not in faces_rawcount_per_decade[newspaper_name]:
                    faces_rawcount_per_decade[newspaper_name][decade] = num_faces #if not there, add the number as the first one
                else:
                    faces_rawcount_per_decade[newspaper_name][decade] += num_faces #if there, add the number to what is allready there



        # now for calculating the percentage of pages with at least 1 face faces per decade:
        # Calculate the total number of pages in the current newspaper
        total_pages = len([filename for filename in os.listdir(full_path) if filename.endswith(".jpg")]) 
        
        
        #add % for each decase, should i loop? :

        # Calculate the percentage
        percentage = (sum(is_face_present) / total_pages) * 100 
        perc_pages_with_faces_per_decade[newspaper_name][decade] = percentage

        
            


    
    
    
    #save results to df
    #for newspaper_name, data in faces_rawcount_per_decade.items():
        #df = pd.DataFrame.from_dict(data, orient='index', columns=['Faces_Count'])
        #df.to_csv(f'../out/{newspaper_name}_faces_per_decade.csv')





# Save results into CSV files:
#for newspaper_name, data in faces_rawcount_per_decade.items():
    #df = pd.DataFrame.from_dict(data, orient='index', columns=['Faces_Count'])
    #df.to_csv(f'../out/{newspaper_name}_faces_per_decade.csv')

#for newspaper_name, data in perc_pages_with_faces_per_decade.items():
    #df = pd.DataFrame.from_dict(data, orient='index', columns=['Percentage'])
    #df.to_csv(f'..out/{newspaper_name}_percentage_per_decade.csv')




TypeError: 'int' object is not iterable

In [5]:
#test, still not what i want

import os
from PIL import Image
from facenet_pytorch import MTCNN

# Initialize MTCNN for face detection:
mtcnn = MTCNN()

# Define folder path
folderpath = "../newspapers_2/"

# Initialize dictionaries to store results:
faces_rawcount_per_decade = {}  # Store raw counts of images with faces per decade for each newspaper
perc_pages_with_faces_per_decade = {}  # Store percentage of pages with faces per decade for each newspaper

# Function to extract decade from filename:
def extract_decade(filename):
    parts = filename.split("-")
    year = int(parts[1])
    decade = (year // 10) * 10
    return decade

# Iterate through each subfolder (newspaper) in folderpath:
for newspaper_folder in os.listdir(folderpath):

    # Construct the path to the newspaper we are looping through 
    full_path = os.path.join(folderpath, newspaper_folder)

    # Check if the path full_path exist as a directory
    if os.path.isdir(full_path):

        # Extract the name of the newspaper from the folder path
        newspaper_name = newspaper_folder.upper()

        # Initialize dictionaries for the current newspaper
        faces_rawcount_per_decade[newspaper_name] = {}
        perc_pages_with_faces_per_decade[newspaper_name] = {}

        # Initialize counter for pages with faces for the current decade
        pages_with_faces = 0

        # Initialize a list to track if faces are present on each page for the current decade
        is_face_present = []

        # Iterate through each file in the subfolder (newspaper):
        for filename in os.listdir(full_path):

            # Construct the full file path
            file_path = os.path.join(full_path, filename)

            # Check if the current item is a file and ends with ".jpg"
            if os.path.isfile(file_path) and filename.endswith(".jpg"):

                # Proceed with processing the image file
                # Open the image file
                img = Image.open(file_path)

                # Extract the decade from the filename
                decade = extract_decade(filename)

                # Detect faces in the image:
                boxes, _ = mtcnn.detect(img)

                # Count faces per decade:
                # Check if any face is detected
                if boxes is not None:
                    num_faces = boxes.shape[0]  # Count the number of detected faces
                else:
                    num_faces = 0  # No faces detected

                # Increment the counter if at least one face is detected on this page
                if num_faces > 0:
                    pages_with_faces += 1
                    is_face_present.append(True)  # Track if faces are present on this page
                else:
                    is_face_present.append(False)  # Track if faces are present on this page

                # Update the count of faces for the current decade
                if decade not in faces_rawcount_per_decade[newspaper_name]:
                    faces_rawcount_per_decade[newspaper_name][decade] = num_faces
                else:
                    faces_rawcount_per_decade[newspaper_name][decade] += num_faces

        # Calculate the total number of pages in the current newspaper
        total_pages = len([filename for filename in os.listdir(full_path) if filename.endswith(".jpg")])

        # Calculate the percentage of pages with at least one face for the current decade
        if total_pages > 0:
            percentage = (pages_with_faces / total_pages) * 100
        else:
            percentage = 0

        # Store the percentage in the perc_pages_with_faces_per_decade dictionary
        perc_pages_with_faces_per_decade[newspaper_name][decade] = percentage

# Now you can use the results stored in faces_rawcount_per_decade and perc_pages_with_faces_per_decade dictionaries
# for further analysis or visualization.


In [7]:
faces_rawcount_per_decade

{'IMP': {2010: 11, 1880: 0},
 'JDG': {1990: 2, 1820: 0},
 'GDL': {1990: 1, 1790: 0}}

In [6]:
perc_pages_with_faces_per_decade

{'IMP': {1880: 37.5}, 'JDG': {1990: 25.0}, 'GDL': {1790: 12.5}}

In [19]:
#test, getting all the deaces %

import os
from PIL import Image
from facenet_pytorch import MTCNN

# Initialize MTCNN for face detection:
mtcnn = MTCNN()

# Define folder path
folderpath = "../newspapers_2/"

# Initialize dictionaries to store results:
faces_rawcount_per_decade = {}  # Store raw counts of images with faces per decade for each newspaper
perc_pages_with_faces_per_decade = {}  # Store percentage of pages with faces per decade for each newspaper

# Function to extract decade from filename:
def extract_decade(filename):
    parts = filename.split("-")
    year = int(parts[1])
    decade = (year // 10) * 10
    return decade


# Iterate through each subfolder (newspaper) in folderpath:
for newspaper_folder in sorted(os.listdir(folderpath)):

    # Construct the path to the newspaper we are looping through 
    full_path = os.path.join(folderpath, newspaper_folder)

    # Check if the path full_path exist as a directory
    if os.path.isdir(full_path):

        # Extract the name of the newspaper from the folder path
        newspaper_name = newspaper_folder.upper()

        # Initialize dictionaries for the current newspaper
        faces_rawcount_per_decade[newspaper_name] = {}
        perc_pages_with_faces_per_decade[newspaper_name] = {}

        # Initialize counter for pages with faces for the current decade
        pages_with_faces = 0 #this is for every newpaper

        # Initialize a list to track if faces are present on each page for the current decade
        #is_face_present = []

        # Iterate through each file in the subfolder (newspaper):
        for filename in sorted(os.listdir(full_path)):

            # Construct the full file path
            file_path = os.path.join(full_path, filename)
            print(file_path)

            # Check if the current item is a file and ends with ".jpg"
            if os.path.isfile(file_path) and filename.endswith(".jpg"):

                # Proceed with processing the image file
                # Open the image file
                img = Image.open(file_path)

                # Extract the decade from the filename
                decade = extract_decade(filename)

                # Detect faces in the image:
                boxes, _ = mtcnn.detect(img)

                # Count faces per decade:
                # Check if any face is detected
                if boxes is not None:
                    num_faces = boxes.shape[0]  # Count the number of detected faces
                else:
                    num_faces = 0  # No faces detected

                # Increment the counter if at least one face is detected on this page
                if num_faces > 0:
                    pages_with_faces += 1 #add to counter for that newspaper 
                    is_face_present = 1 # Track if faces are present on this page
                else:
                    is_face_present = 0  # Track if faces are present on this page

                # Update the count of faces for the current decade
                if decade not in faces_rawcount_per_decade[newspaper_name]:
                    faces_rawcount_per_decade[newspaper_name][decade] = num_faces
                else:
                    faces_rawcount_per_decade[newspaper_name][decade] += num_faces

                #can i Update the count of is_face_present  for the current decade
                if decade not in perc_pages_with_faces_per_decade[newspaper_name]:
                    perc_pages_with_faces_per_decade[newspaper_name][decade] = is_face_present
                else:
                    perc_pages_with_faces_per_decade[newspaper_name][decade] += is_face_present

         

        # Calculate the total number of pages in the current newspaper
        #total_pages = len([filename for filename in os.listdir(full_path) if filename.endswith(".jpg")])

        # Calculate the percentage of pages with at least one face for the current decade
        #if total_pages > 0:
            #percentage = (pages_with_faces / total_pages) * 100
        #else:
            #percentage = 0

        # Store the percentage in the perc_pages_with_faces_per_decade dictionary
        #perc_pages_with_faces_per_decade[newspaper_name][decade] = percentage

# Now you can use the results stored in faces_rawcount_per_decade and perc_pages_with_faces_per_decade dictionaries
# for further analysis or visualization.


../newspapers_2/GDL/GDL-1798-02-05-a-p0001.jpg
../newspapers_2/GDL/GDL-1798-02-05-a-p0002.jpg
../newspapers_2/GDL/GDL-1798-02-05-a-p0003.jpg
../newspapers_2/GDL/GDL-1798-02-05-a-p0004.jpg
../newspapers_2/GDL/GDL-1997-08-09-a-p0029.jpg
../newspapers_2/GDL/GDL-1997-08-09-a-p0030.jpg
../newspapers_2/GDL/GDL-1997-08-09-a-p0031.jpg
../newspapers_2/GDL/GDL-1997-08-09-a-p0032.jpg
../newspapers_2/IMP/IMP-1882-05-04-a-p0001.jpg
../newspapers_2/IMP/IMP-1882-05-04-a-p0002.jpg
../newspapers_2/IMP/IMP-1882-05-04-a-p0003.jpg
../newspapers_2/IMP/IMP-1882-05-04-a-p0004.jpg
../newspapers_2/IMP/IMP-2017-12-30-a-p0025.jpg
../newspapers_2/IMP/IMP-2017-12-30-a-p0026.jpg
../newspapers_2/IMP/IMP-2017-12-30-a-p0027.jpg
../newspapers_2/IMP/IMP-2017-12-30-a-p0028.jpg
../newspapers_2/JDG/JDG-1826-02-16-a-p0001.jpg
../newspapers_2/JDG/JDG-1826-02-16-a-p0002.jpg
../newspapers_2/JDG/JDG-1826-02-16-a-p0003.jpg
../newspapers_2/JDG/JDG-1826-02-16-a-p0004.jpg
../newspapers_2/JDG/JDG-1998-02-27-a-p0041.jpg
../newspapers

In [20]:
faces_rawcount_per_decade

{'GDL': {1790: 0, 1990: 1},
 'IMP': {1880: 0, 2010: 11},
 'JDG': {1820: 0, 1990: 2}}

In [21]:
perc_pages_with_faces_per_decade

{'GDL': {1790: 0, 1990: 1},
 'IMP': {1880: 0, 2010: 3},
 'JDG': {1820: 0, 1990: 2}}

In [10]:
is_face_present

0

In [ ]:
#method 2:

#loop through each subfolder in folderpath

    #extract name of newspaper:
    #newpaper_name = in subfolder the first charachters until '-' 

    #make an empty list to store decades:
    #decades = []

    #loop thoguh each file in subfolder:
        #for file in subfolder:

        # get decade:

        ##make an empty list to store decades:
        ##decades = []
        
        ##get filename:
        ##filename = what comes after the last / in file

        ##between the first '-' and second '-' in filename extract 3 first numbers:
        ##3_numbers = 


        #add unique instances to decades list: 
        #if 3_numbers is not allready in decades, add it to the list
            #else skip
        
        #for each decade in decades: 
            #take the filenames where 5-8th charachter matches the decase 

            #load in image:
            #image_path= Image.open(path_to_image)



            # make empty varibale for counting number of faces:
            #faces_on_page_count= 0


            #detect faces 
            #boxes, _ = mtcnn.detect(img)
        

            #if shape of boxes == Nonce
                #add 0 to faces_on_page

                #else
                    #number_to_add = boxes.shape[0]
                    #add number_to_add to faces_on_page_count
            
        

        


In [ ]:
#plot
#x = decade, y= percentage, titel= name of newspaper